In [45]:
import json
import keras_tuner as kt
import numpy as np
import pandas as pd
import plotly.express as px
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

---
### Data Reading
---

In [46]:
data = np.load('../data/training_data.npz', allow_pickle=True)

train_set = data['train_set']
test_set = data['test_set']
val_set = data['val_set']

In [47]:
def create_dataframe(set):
    df_ = pd.DataFrame([], columns=['x', 'y'])	
    for i in range(len(set)):
        df_ = pd.concat([df_, pd.DataFrame(
            {'reading': [set[i]['x']],
            'posture': set[i]['y']}
        )])
    return df_

In [48]:
df_train = create_dataframe(train_set)
df_test = create_dataframe(test_set)
df_val = create_dataframe(val_set)

print(df_train.shape, df_test.shape, df_val.shape)

(108, 4) (54, 4) (27, 4)


In [49]:
X_train = df_train['reading']
y_train = df_train['posture']
X_val = df_val['reading']
y_val = df_val['posture']
X_test = df_test['reading']
y_test = df_test['posture']
# One-hot encode the labels
y_train = pd.get_dummies(y_train).values
y_val = pd.get_dummies(y_val).values
y_test = pd.get_dummies(y_test).values

In [50]:
X_train = np.array([np.array(x) for x in X_train])
X_train = np.expand_dims(X_train, -1)
X_val = np.array([np.array(x) for x in X_val])
X_val = np.expand_dims(X_val, -1)
X_test = np.array([np.array(x) for x in X_test])
X_test = np.expand_dims(X_test, -1)
y_train = np.array([np.array(x) for x in y_train])
y_test = np.array([np.array(x) for x in y_test])
y_val = np.array([np.array(x) for x in y_val])
# Convert the data to tensors
X_train = tf.convert_to_tensor(X_train)
X_val = tf.convert_to_tensor(X_val)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_val = tf.convert_to_tensor(y_val)
y_test = tf.convert_to_tensor(y_test)

---
### Model Definition
---

In [51]:
def model_builder(hp):
    hp_channels_1 = hp.Int('channels_1', min_value=4, max_value=16, step=4)
    hp_channels_2 = hp.Int('channels_2', min_value=4, max_value=8, step=4)
    hp_dense_1 = hp.Int('dense_1', min_value=16, max_value=64, step=16)
    hp_dense_2 = hp.Int('dense_2', min_value=16, max_value=32, step=16)
    model = keras.Sequential(
        [
            layers.Conv2D(hp_channels_1, kernel_size=(3, 3), activation='relu', padding='same', input_shape=X_train.shape[1:]),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(hp_channels_2, kernel_size=(3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),  # Flatten the output of the CNN
            layers.Dense(hp_dense_1, activation='relu'),
            layers.Dense(hp_dense_2, activation='relu'),
            layers.Dense(3, activation='softmax'),
        ]
    )
    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'],
    )
    return model

In [52]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=20,
    min_delta=0.001,
    restore_best_weights=True,
)
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    patience=5
)

In [53]:
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=500,
    max_retries_per_trial=2,
    factor=3,
    directory='hyperband-laguardia',
    project_name='posture',
)

Reloading Tuner from hyperband-laguardia\posture\tuner0.json


---
### Model Training
---

In [54]:
tuner.search(X_train, y_train, validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr])

Trial 24 Complete [00h 00m 04s]
val_accuracy: 0.3333333432674408

Best val_accuracy So Far: 0.6666666865348816
Total elapsed time: 00h 01m 49s

Search: Running Trial #25

Value             |Best Value So Far |Hyperparameter
4                 |12                |channels_1
8                 |8                 |channels_2
64                |32                |dense_1
16                |16                |dense_2
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2


KeyboardInterrupt: 

In [ ]:
tuner.results_summary()

---
### Model Evaluation
---

In [ ]:
best_model = tuner.get_best_models(1)[0]
best_model.save('best_model_hyperband.keras')

In [ ]:
best_model.evaluate(X_test, y_test)